# Momentum strategy backtester. 

In this notebook I look to implement and backtest the momentum strategy from this paper https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2244633 .

In [31]:
from backtester import getdata
import yfinance as yf
import pandas   as pd
import numpy    as np
import random
from numba import jit, njit, types, vectorize, prange
import plotly.express as plt



ImportError: cannot import name 'getdata' from 'backtester' (/Users/sufyansaleem/Ts-backtester/Trading-strategy-backtester/backtester.py)

To implament the strategy from the paper we will 

In [ ]:
getdata('BTC-USD','30y','Close','1D')


NameError: name 'getdata' is not defined